# Extract symbols
This notebook will extarct all symbols and write it to the database

In [23]:
# Import libraries
import binance
import pandas as pd
import ccxt
import logging
import datetime
import tqdm
import time
import os
import traceback
import sqlite3
import pdcast as pdc
from dotenv import load_dotenv


In [3]:
# Load environment variables
_ = load_dotenv()
working_dir = os.getenv("WORKING_DIR")

# Change to working dir
os.chdir(working_dir)

In [4]:
def extract_symbols():
    exchange = ccxt.binance(
        {
            "rateLimit": 2000,
            "enableRateLimit": True,
            "verbose": False,
        }
    )
    data = exchange.load_markets()
    df = pd.DataFrame(data).T.reset_index()
    df = df.drop(columns=['precision','limits','info'])
    return df

In [27]:
df = extract_symbols()

In [28]:
df_downcast = pdc.downcast(df)

In [36]:
sql_text = pd.io.sql.get_schema(df_downcast, "DimSymbols")

In [37]:
print(sql_text)

CREATE TABLE "DimSymbols" (
"index" TEXT,
  "id" TEXT,
  "symbol" TEXT,
  "base" TEXT,
  "quote" TEXT,
  "baseId" TEXT,
  "quoteId" TEXT,
  "active" TEXT,
  "type" TEXT,
  "linear" TEXT,
  "inverse" TEXT,
  "spot" TEXT,
  "swap" TEXT,
  "future" TEXT,
  "option" TEXT,
  "margin" TEXT,
  "contract" TEXT,
  "contractSize" TEXT,
  "expiry" TEXT,
  "expiryDatetime" TEXT,
  "optionType" INTEGER,
  "strike" INTEGER,
  "settle" TEXT,
  "settleId" TEXT,
  "percentage" TEXT,
  "feeSide" TEXT,
  "tierBased" TEXT,
  "taker" TEXT,
  "maker" TEXT,
  "lowercaseId" TEXT
)


In [42]:
sql = """
CREATE TABLE "DimSymbols" (
"pk_symbols" INTEGER PRIMARY KEY,
"index" TEXT,
  "id" TEXT,
  "symbol" TEXT,
  "base" TEXT,
  "quote" TEXT,
  "baseId" TEXT,
  "quoteId" TEXT,
  "active" TEXT,
  "type" TEXT,
  "linear" TEXT,
  "inverse" TEXT,
  "spot" TEXT,
  "swap" TEXT,
  "future" TEXT,
  "option" TEXT,
  "margin" TEXT,
  "contract" TEXT,
  "contractSize" TEXT,
  "expiry" TEXT,
  "expiryDatetime" TEXT,
  "optionType" INTEGER,
  "strike" INTEGER,
  "settle" TEXT,
  "settleId" TEXT,
  "percentage" TEXT,
  "feeSide" TEXT,
  "tierBased" TEXT,
  "taker" TEXT,
  "maker" TEXT,
  "lowercaseId" TEXT
)
"""

with sqlite3.connect("data/database.db") as conn:
    conn.execute("drop table if exists DimSymbols")
    conn.execute(sql)
    conn.commit()


In [43]:
# Write data to database
with sqlite3.connect(database="./data/database.db") as conn:
    logging.info("Writing to database")
    df_downcast.to_sql("DimSymbols", conn, if_exists="append", index=False)